In [1]:
import  pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv(r'C:\Users\DELL PC\Desktop\ML\DL\useful_data\insurance.csv')
df.head()

,Age,Previously_Insured,Response
0,44,0,1
1,76,0,0
2,47,0,1
3,21,1,0
4,29,1,0


In [3]:
import tensorflow as tf

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df[['Age','Previously_Insured']],df.Response,test_size=.3,random_state=58)

In [8]:
X_train_scaled=X_train.copy()                   #Scaling
X_train_scaled['Age']=X_train_scaled['Age']/100

X_test_scaled=X_test.copy()
X_test_scaled['Age']=X_test_scaled['Age']/100

In [9]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train_scaled,y_train,epochs=500)


#kernel_initilizer --> initial weights for all inputs
#bias_initializer --> initial bias(or intercept)

Epoch 1/500
27/27 [==============================] - 3s 4ms/step - loss: 1.1539 - accuracy: 0.1274
Epoch 2/500
27/27 [==============================] - 0s 2ms/step - loss: 1.1244 - accuracy: 0.1274
Epoch 3/500
27/27 [==============================] - 0s 3ms/step - loss: 1.0956 - accuracy: 0.1274
Epoch 4/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0677 - accuracy: 0.1274
Epoch 5/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0406 - accuracy: 0.1274
Epoch 6/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0138 - accuracy: 0.1274
Epoch 7/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9878 - accuracy: 0.1321
Epoch 8/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9628 - accuracy: 0.1893
Epoch 9/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9384 - accuracy: 0.2202
Epoch 10/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9147 - accuracy: 0.2310
Epoch 11/

In [10]:
model.evaluate(X_test_scaled,y_test)

12/12 [==============================] - 0s 2ms/step - loss: 0.2896 - accuracy: 0.8837


[0.2896100580692291, 0.8836565017700195]

In [14]:
model.predict(X_test_scaled)

array([[0.24693412],
       [0.22561523],
       [0.00132242],
       [0.00132242],
       [0.22896364],
       [0.22896364],
       [0.00132242],
       [0.00132748],
       [0.21772131],
       [0.00136337],
       [0.00131238],
       [0.00130248],
       [0.24551868],
       [0.00135821],
       [0.00133258],
       [0.00132748],
       [0.23098943],
       [0.23852399],
       [0.24340582],
       [0.22033024],
       [0.00130746],
       [0.2350784 ],
       [0.00140554],
       [0.2303128 ],
       [0.23991194],
       [0.00131238],
       [0.00132748],
       [0.23166743],
       [0.00136852],
       [0.22628212],
       [0.21837142],
       [0.00131741],
       [0.23302767],
       [0.22164315],
       [0.2350784 ],
       [0.2378321 ],
       [0.21902299],
       [0.00132748],
       [0.21772131],
       [0.22164315],
       [0.00143254],
       [0.00132242],
       [0.24130535],
       [0.2378321 ],
       [0.00134271],
       [0.00130746],
       [0.23234683],
       [0.001

In [19]:
X_test_scaled

,Age,Previously_Insured
289,0.64,0
754,0.33,0
553,0.24,1
286,0.24,1
1041,0.38,0
...,...,...
949,0.46,0
1139,0.27,1
926,0.26,1
129,0.27,1


In [16]:
coef,intercept=model.get_weights()
coef,intercept

(array([[ 0.38131204],
        [-5.359377  ]], dtype=float32),
 array([-1.3590709], dtype=float32))

In [17]:
# to show how neuron works if we know weights and bias

def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [18]:
def prediction_function(age,affordibility):
    weighted_sum=coef[0]*age+coef[1]*affordibility+intercept
    return sigmoid(weighted_sum)

In [20]:
prediction_function(.64,0)

0.24693409977490557

In [21]:
# to implement gradient descent from SCRATCH

def log_loss(y_true,y_predicted):
    epsilon=1e-15
    y_predicted_new=[max(i,epsilon) for i in y_predicted]
    y_predicted_new=[min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new=np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [26]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [46]:
def gradient_descent(age,affordability,y_true,epochs): #help to find optimal value of 'w' and 'b'
    # w1,w2,bias
    w1=w2=1
    bias=0
    learning_rate=.5
    n=len(age)

    for i in range(epochs):
        weighted_sum=w1*age+w2*affordability+bias
        y_predicted=sigmoid_numpy(weighted_sum)

        loss=log_loss(y_true,y_predicted)

        w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        # w1d=np.mean(age*(y_predicted-y_true))

        w2d=(1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
        # w2d=np.mean(affordability*(y_predicted-y_true))
        
        bias_d=np.mean(y_predicted-y_true)

        w1=w1-learning_rate*w1d
        w2=w2-learning_rate*w2d
        bias=bias-learning_rate*bias_d

        print(f'Epoch:{i+1},  w1:{w1}, w2:{w2}, loss:{loss}')
        
    return w1,w2,bias


In [48]:
gradient_descent(X_train_scaled['Age'],X_train_scaled['Previously_Insured'],y_train,500)

Epoch:1,  w1:0.8931722670815012, w2:0.8213180612961857, loss:1.1677376027035697
Epoch:2,  w1:0.8028246159095046, w2:0.6635539935874049, loss:0.94028745191894
Epoch:3,  w1:0.7280887087052925, w2:0.5274471114199698, loss:0.7750514159481139
Epoch:4,  w1:0.666936962219357, w2:0.41118961325601217, loss:0.6593809744022665
Epoch:5,  w1:0.6169686724477094, w2:0.3117208342036162, loss:0.5794929527547031
Epoch:6,  w1:0.5759577362427858, w2:0.2258627689088409, loss:0.5240112878725864
Epoch:7,  w1:0.5420644337483611, w2:0.15086343700318272, loss:0.48482859238661014
Epoch:8,  w1:0.5138456116186538, w2:0.0845214587368141, loss:0.4565526797359132
Epoch:9,  w1:0.49019065096805664, w2:0.025131656155278653, loss:0.4356799618940039
Epoch:10,  w1:0.4702466163468452, w2:-0.028615109900178008, loss:0.4199308380962959
Epoch:11,  w1:0.4533542681950615, w2:-0.0777266170319859, loss:0.40780195716701034
Epoch:12,  w1:0.4389988037598394, w2:-0.12298506912225987, loss:0.398283236329726
Epoch:13,  w1:0.426773441410

(0.6862607514914825, -3.337658845493573, -1.5382103556936044)